In [1]:
import pandas as pd
import numpy as np

from lol_fandom import SITE
from lol_fandom import get_leagues, get_tournaments
from lol_fandom import get_scoreboard_games, get_scoreboard_players
from lol_fandom import from_response

pd.set_option('display.max_columns', None)

In [2]:
TARGET_LEAGUES = [
    'LTC', 'LCK',
    'LPL',
    'EU LCS', 'LEC',
    'NA LCS', 'LCS',
    'GPL', 'LST',
    'LLA', 'CLS', 'LLN',
    'LCO', 'OPL',
    'LCL',
    'LJL',
    'LMS', 'PCS',
    'VCS',
    'CBLOL', 
    'TCL',
    
    'MSI',
    'WCS',
]

In [3]:
def get_new_id(teams):
    id_list = sorted(teams['team_id'].unique())

    if len(id_list) == 0:
        return 1
    prev = id_list[0]
    for id in id_list[1:]:
        if prev + 1 != id:
            return prev + 1
        prev = id
    return prev + 1

In [4]:
leagues = get_leagues(where='L.Level="Primary" and L.IsOfficial="Yes"')
leagues

,League,League Short,Region,Level,IsOfficial
0,2015 International Wildcard Tournament,2015 IWCT,International,Primary,Yes
1,2016 International Wildcard Qualifier,IWCQ,International,Primary,Yes
2,Circuit Brazilian League of Legends,CBLOL,Brazil,Primary,Yes
3,Copa Latinoamérica Sur,CLS,LAS,Primary,Yes
4,DreamHack,DH,International,Primary,Yes
5,Europe League Championship Series,EU LCS,Europe,Primary,Yes
6,Garena Premier League,GPL,SEA,Primary,Yes
7,International Wildcard Invitational,2015 IWCI,International,Primary,Yes
8,League of Legends Championship Series,LCS,North America,Primary,Yes
9,League of Legends SEA Tour,LST,SEA,Primary,Yes


In [5]:
lst = []
for league in TARGET_LEAGUES:
    if league not in leagues['League Short'].values:
        lst.append(league)
lst

[]

In [6]:
teams = pd.read_csv('./csv/teams_id.csv')

for league in TARGET_LEAGUES:
    print(league)
    tournaments = get_tournaments(where=f'L.League_Short="{league}"')
    tournaments = tournaments.sort_values(
        by=['Year', 'DateStart', 'Date']
    ).reset_index(drop=True)

    for page in tournaments['OverviewPage']:
        print(f'\t{page}')
        scoreboard_games = get_scoreboard_games(
            where=f'T.OverviewPage="{page}"'
        )
        if scoreboard_games is not None:
            scoreboard_games = scoreboard_games.sort_values(
                by='DateTime UTC'
            ).reset_index(drop=True)
            team_names = scoreboard_games[['Team1', 'Team2']].unstack().unique()
            names = []
            for name in team_names:
                if name not in teams['team'].values:
                    names.append(name)
            if len(names) > 0:
                names = sorted(names)
                break
    if len(names) > 0:
        print(scoreboard_games['OverviewPage'].unique()[0])
        print(names)
        break

LTC
	Champions/2012 Season/Spring
	Champions/2012 Season/Summer
	Champions/2013 Season/Winter Qualifiers
	Champions/2013 Season/Winter
	Champions/2013 Season/Spring Qualifiers
	Champions/2013 Season/Spring
	Champions/2013 Season/Summer Qualifiers
	Champions/2013 Season/Summer
	Champions/2014 Season/Winter Qualifiers
	Champions/2014 Season/Winter Season
	Champions/2014 Season/Spring Qualifiers
	Champions/2014 Season/Spring Season
	Champions/2014 Season/Summer Qualifiers
	Champions/2014 Season/Summer Season
	Champions/2015 Season/Spring Qualifiers
	Champions/2015 Season/Spring Preseason
	Champions/2015 Season/Spring Season
	Champions/2015 Season/Spring Playoffs
	Champions/2015 Season/Summer Promotion
	Champions/2015 Season/Summer Season
	Champions/2015 Season/Summer Playoffs
	Azubu LoL The Champions All Stars
LCK
	LCK/2016 Season/Spring Promotion
	LCK/2016 Season/Spring Season
	LCK/2016 Season/Spring Playoffs
	LCK/2016 Season/Summer Promotion
	LCK/2016 Season/Summer Season
	LCK/2016 Seas

In [518]:
teams.loc[teams['team'].str.contains('Afro')]

,team,team_id
531,Team Afro,587


In [529]:
teams = pd.concat(
    [
        teams,
        pd.Series({'team': names.pop(0), 'team_id': get_new_id(teams)}).to_frame().T
        # pd.Series({'team': names.pop(0), 'team_id': 587}).to_frame().T
    ],
    ignore_index=True
)
print(names)
teams

[]


,team,team_id
0,100 Thieves,253
1,17 Academy,586
2,1UP Gaming,202
3,2144 Danmu Gaming,91
4,269 Gaming,506
...,...,...
694,paiN Gaming,262
695,team propertybb2,139
696,vVv Gaming,213
697,yoe Flash Wolves,459


In [530]:
names

[]

In [531]:
teams = teams.sort_values(by='team').reset_index(drop=True)
teams

,team,team_id
0,100 Thieves,253
1,17 Academy,586
2,1UP Gaming,202
3,2144 Danmu Gaming,91
4,269 Gaming,506
...,...,...
694,n!faculty,168
695,paiN Gaming,262
696,team propertybb2,139
697,vVv Gaming,213


In [532]:
teams.to_csv('teams_id.csv', index=False)

In [7]:
leagues = get_leagues(where='L.Level="Primary" and L.IsOfficial="Yes"')
leagues

,League,League Short,Region,Level,IsOfficial
0,2015 International Wildcard Tournament,2015 IWCT,International,Primary,Yes
1,2016 International Wildcard Qualifier,IWCQ,International,Primary,Yes
2,Circuit Brazilian League of Legends,CBLOL,Brazil,Primary,Yes
3,Copa Latinoamérica Sur,CLS,LAS,Primary,Yes
4,DreamHack,DH,International,Primary,Yes
5,Europe League Championship Series,EU LCS,Europe,Primary,Yes
6,Garena Premier League,GPL,SEA,Primary,Yes
7,International Wildcard Invitational,2015 IWCI,International,Primary,Yes
8,League of Legends Championship Series,LCS,North America,Primary,Yes
9,League of Legends SEA Tour,LST,SEA,Primary,Yes


In [8]:
# for year in range(2011, 2023):
for year in range(2023, 2024):
    tournaments = pd.DataFrame()
    for league in TARGET_LEAGUES:
        t = get_tournaments(where=f'L.League_Short="{league}" and T.Year={year}')
        tournaments = pd.concat([tournaments, t])
    tournaments = tournaments.sort_values(
        by=['Year', 'DateStart', 'Date']
    ).reset_index(drop=True)

    tournaments.to_csv(f'./csv/tournaments/{year}_tournaments.csv', index=False)
    print(year, tournaments.shape)

2023 (20, 17)


In [9]:
# for year in range(2011, 2023):
for year in range(2023, 2024):
    tournaments = pd.read_csv(f'./csv/tournaments/{year}_tournaments.csv')
    print(year, tournaments.shape)
    scoreboard_games = pd.DataFrame()
    for page in tournaments['OverviewPage']:
        sg = get_scoreboard_games(where=f'T.OverviewPage="{page}"')
        if sg is None:
            print(f'{year}\t{page:35}\tdrop')
            tournaments.drop(tournaments.loc[tournaments['OverviewPage'] == page].index, inplace=True)
            continue
        league = tournaments.loc[tournaments['OverviewPage'] == page, 'League'].iloc[0]
        league = leagues.loc[leagues['League'] == league, 'League Short'].iloc[0]
        sg['League'] = league
        sg = sg.sort_values(
            by='DateTime UTC'
        ).reset_index(drop=True)
        print(f'\t{sg.shape}\t{page}')
        scoreboard_games = pd.concat([scoreboard_games, sg])
    scoreboard_games.to_csv(f'./csv/scoreboard_games/{year}_scoreboard_games.csv', index=False)
    print(year, scoreboard_games.shape)
    tournaments.to_csv(f'./csv/tournaments/{year}_tournaments.csv', index=False)
    print(year, tournaments.shape, '\n\n')

2023 (20, 17)
	(7, 37)	LLA/2023 Season/Opening Promotion
	(11, 37)	LPL/2023 Season/Spring Season
2023	LCK/2023 Season/Spring Season      	drop
2023	LEC/2023 Season/Winter Season      	drop
2023	CBLOL/2023 Season/Split 1          	drop
2023	LCO/2023 Season/Split 1            	drop
2023	LLA/2023 Season/Opening Season     	drop
2023	LCS/2023 Season/Spring Season      	drop
2023	TCL/2023 Season/Winter Season      	drop
2023	LJL/2023 Season/Spring Season      	drop
2023	PCS/2023 Season/Spring Season      	drop
2023	LEC/2023 Season/Winter Groups      	drop
2023	LEC/2023 Season/Winter Playoffs    	drop
2023	LLA/2023 Season/Opening Playoffs   	drop
2023	LCK/2023 Season/Spring Playoffs    	drop
2023	2023 Mid-Season Invitational       	drop
2023	LPL/2023 Season/Spring Playoffs    	drop
2023	LEC/2023 Season/Season Finals      	drop
2023	LJL/2023 Season/Spring Playoffs    	drop
2023	VCS/2023 Season/Spring Season      	drop
2023 (18, 37)
2023 (2, 17) 




In [17]:
year = 2023
tournaments = pd.DataFrame()
for league in TARGET_LEAGUES:
    t = get_tournaments(where=f'L.League_Short="{league}" and T.Year={year}')
    tournaments = pd.concat([tournaments, t])
tournaments = tournaments.sort_values(
    by=['Year', 'DateStart', 'Date']
).reset_index(drop=True)


tournaments

,Name,OverviewPage,DateStart,Date,League,Region,EventType,StandardName,Split,SplitNumber,TournamentLevel,IsQualifier,IsPlayoffs,IsOfficial,Year,DateStart__precision,Date__precision
0,LLA 2023 Opening Promotion,LLA/2023 Season/Opening Promotion,2022-09-10,2022-09-11,Liga Latinoamerica,Latin America,Offline,LLA 2023 Opening Promotion,None,None,Primary,1,0,1,2023,1,1
1,LPL 2023 Spring,LPL/2023 Season/Spring Season,2023-01-14,2023-03-26,Tencent LoL Pro League,China,Offline,LPL 2023 Spring,Spring,1,Primary,0,0,1,2023,1,1
2,LCK 2023 Spring,LCK/2023 Season/Spring Season,2023-01-18,2023-03-19,LoL Champions Korea,Korea,None,LCK 2023 Spring,Spring,1,Primary,0,0,1,2023,1,1
3,LEC 2023 Winter Season,LEC/2023 Season/Winter Season,2023-01-21,2023-02-06,LoL EMEA Championship,EMEA,Offline,LEC 2023 Winter Season,Winter,1,Primary,0,0,1,2023,1,1
4,CBLOL 2023 Split 1,CBLOL/2023 Season/Split 1,2023-01-21,2023-03-19,Circuit Brazilian League of Legends,Brazil,Online,CBLOL 2023 Split 1,Split 1,1,Primary,0,0,1,2023,1,1
5,LCO 2023 Split 1,LCO/2023 Season/Split 1,2023-01-23,NaT,LoL Circuit Oceania,PCS,None,LCO 2023 Split 1,Split 1,1,Primary,0,0,1,2023,1,None
6,LLA 2023 Opening,LLA/2023 Season/Opening Season,2023-01-24,2023-03-08,Liga Latinoamerica,Latin America,Offline,LLA 2023 Opening,Opening,1,Primary,0,0,1,2023,1,1
7,LCS 2023 Spring,LCS/2023 Season/Spring Season,2023-01-26,2023-03-17,League of Legends Championship Series,North America,None,LCS 2023 Spring,Spring,1,Primary,0,0,1,2023,1,1
8,TCL 2023 Winter,TCL/2023 Season/Winter Season,2023-01-26,NaT,Turkish Championship League,EMEA,Online,TCL 2023 Winter,Winter,1,Primary,0,0,1,2023,1,None
9,LJL 2023 Spring,LJL/2023 Season/Spring Season,2023-01-28,NaT,LoL Japan League,Japan,Online,LJL 2023 Spring,Spring,1,Primary,0,0,1,2023,1,None


In [18]:
scoreboard_games = pd.DataFrame()
for page in tournaments['OverviewPage']:
    sg = get_scoreboard_games(where=f'T.OverviewPage="{page}"')
    if sg is None:
        print(f'{year}\t{page:35}\tdrop')
        tournaments.drop(
            tournaments.loc[tournaments['OverviewPage'] == page].index,
            inplace=True
        )
        continue
    league = tournaments.loc[tournaments['OverviewPage'] == page, 'League'].iloc[0]
    league = leagues.loc[leagues['League'] == league, 'League Short'].iloc[0]
    sg['League'] = league
    sg = sg.sort_values(
        by='DateTime UTC'
    ).reset_index(drop=True)
    print(f'\t{sg.shape}\t{page}')
    scoreboard_games = pd.concat([scoreboard_games, sg])
print(year, scoreboard_games.shape)
print(year, tournaments.shape, '\n\n')

	(7, 37)	LLA/2023 Season/Opening Promotion
	(3, 37)	LPL/2023 Season/Spring Season
2023	LCK/2023 Season/Spring Season      	drop
2023	LEC/2023 Season/Winter Season      	drop
2023	CBLOL/2023 Season/Split 1          	drop
2023	LCO/2023 Season/Split 1            	drop
2023	LLA/2023 Season/Opening Season     	drop
2023	LCS/2023 Season/Spring Season      	drop
2023	TCL/2023 Season/Winter Season      	drop
2023	LJL/2023 Season/Spring Season      	drop
2023	PCS/2023 Season/Spring Season      	drop
2023	LEC/2023 Season/Winter Groups      	drop
2023	LEC/2023 Season/Winter Playoffs    	drop
2023	LLA/2023 Season/Opening Playoffs   	drop
2023	LCK/2023 Season/Spring Playoffs    	drop
2023	2023 Mid-Season Invitational       	drop
2023	LPL/2023 Season/Spring Playoffs    	drop
2023	LEC/2023 Season/Season Finals      	drop
2023	VCS/2023 Season/Spring Season      	drop
2023 (10, 37)
2023 (2, 17) 




In [19]:
scoreboard_games

,OverviewPage,Team1,Team2,WinTeam,LossTeam,DateTime UTC,Team1Score,Team2Score,Winner,Gamelength,Gamelength Number,Team1Bans,Team2Bans,Team1Picks,Team2Picks,Team1Players,Team2Players,Team1Dragons,Team2Dragons,Team1Barons,Team2Barons,Team1Towers,Team2Towers,Team1Gold,Team2Gold,Team1Kills,Team2Kills,Team1RiftHeralds,Team2RiftHeralds,Team1Inhibitors,Team2Inhibitors,Patch,GameId,MatchId,RiotGameId,DateTime UTC__precision,League
0,LLA/2023 Season/Opening Promotion,Incubus,XTEN Esports,XTEN Esports,Incubus,2022-09-10 20:09:00,0,1,2,29:46,29.766667,"Kalista,Gnar,LeBlanc,Sivir,Tahm Kench","Vi,Lucian,Renekton,Zeri,Aphelios","Gwen,Poppy,Taliyah,Jinx,Lulu","Fiora,Sejuani,Azir,Draven,Nautilus","JuanDeDios,Shall (Jorge Mendoza),Piqueos,Scena...","Zerito,Seize,Keine,Krim,Bardito",2.0,2.0,0.0,2.0,1.0,9.0,48256.0,60130.0,7.0,13.0,1.0,1.0,0.0,1.0,12.16,LLA/2023 Season/Opening Promotion_Qualifiers_1_1,LLA/2023 Season/Opening Promotion_Qualifiers_1,None,0,LLA
1,LLA/2023 Season/Opening Promotion,XTEN Esports,Incubus,XTEN Esports,Incubus,2022-09-10 20:59:00,2,0,1,26:38,26.633333,"Vi,Lucian,Renekton,Jarvan IV,Wukong","Kalista,Fiora,Azir,Tahm Kench,Lulu","Ornn,Sejuani,Sylas,Jinx,Amumu","Gwen,Lee Sin,LeBlanc,Aphelios,Renata Glasc","Zerito,Seize,Keine,Krim,Bardito","JuanDeDios,Shall (Jorge Mendoza),Piqueos,Scena...",2.0,1.0,1.0,0.0,8.0,1.0,55042.0,44598.0,20.0,10.0,2.0,0.0,1.0,0.0,12.16,LLA/2023 Season/Opening Promotion_Qualifiers_1_2,LLA/2023 Season/Opening Promotion_Qualifiers_1,None,0,LLA
2,LLA/2023 Season/Opening Promotion,Incubus,XTEN Esports,XTEN Esports,Incubus,2022-09-10 21:49:00,0,3,2,31:56,31.933333,"Kalista,Fiora,LeBlanc,Lulu,Irelia","Vi,Lucian,Renekton,Yuumi,Braum","Aatrox,Jarvan IV,Azir,Zeri,Bard","Camille,Sejuani,Sylas,Sivir,Renata Glasc","JuanDeDios,Shall (Jorge Mendoza),Piqueos,Scena...","Zerito,Seize,Keine,Krim,Bardito",1.0,3.0,0.0,2.0,0.0,11.0,50902.0,65286.0,5.0,14.0,0.0,2.0,0.0,4.0,12.16,LLA/2023 Season/Opening Promotion_Qualifiers_1_3,LLA/2023 Season/Opening Promotion_Qualifiers_1,None,0,LLA
3,LLA/2023 Season/Opening Promotion,Globant Emerald,The Kings,The Kings,Globant Emerald,2022-09-11 20:08:00,0,1,2,30:36,30.600000,"Trundle,Azir,Yuumi,Gangplank,Ahri","Kalista,Lucian,Sejuani,Sylas,Lissandra","Ornn,Vi,Akali,Zeri,Lulu","Aatrox,Wukong,Taliyah,Sivir,Nautilus","Nate,MrLemon,WARANGELUS,Whitelotus,Newbie","Destroy (Yoon Jeong-min),MATAZ,Rainbow (Kim So...",0.0,4.0,0.0,1.0,2.0,9.0,47461.0,58407.0,6.0,15.0,2.0,0.0,0.0,1.0,12.16,LLA/2023 Season/Opening Promotion_Qualifiers_2_1,LLA/2023 Season/Opening Promotion_Qualifiers_2,None,0,LLA
4,LLA/2023 Season/Opening Promotion,The Kings,Globant Emerald,The Kings,Globant Emerald,2022-09-11 21:01:00,2,0,1,32:47,32.783333,"Kalista,Lucian,Senna,Akali,Ornn","Trundle,Azir,Sejuani,Taliyah,Gangplank","Renekton,Jarvan IV,Sylas,Zeri,Yuumi","Aatrox,Wukong,LeBlanc,Sivir,Nautilus","Destroy (Yoon Jeong-min),MATAZ,Rainbow (Kim So...","Nate,MrLemon,WARANGELUS,Whitelotus,Newbie",3.0,1.0,1.0,0.0,9.0,3.0,61630.0,53630.0,19.0,10.0,1.0,1.0,2.0,0.0,12.16,LLA/2023 Season/Opening Promotion_Qualifiers_2_2,LLA/2023 Season/Opening Promotion_Qualifiers_2,None,0,LLA
5,LLA/2023 Season/Opening Promotion,Globant Emerald,The Kings,Globant Emerald,The Kings,2022-09-11 21:57:00,1,2,1,45:26,45.433333,"Trundle,Azir,Taliyah,Lulu,Gangplank","Kalista,Lucian,Sejuani,LeBlanc,Sylas","Ornn,Vi,Viktor,Sivir,Yuumi","Aatrox,Wukong,Ahri,Zeri,Rakan","Nate,MrLemon,WARANGELUS,Whitelotus,Newbie","Destroy (Yoon Jeong-min),MATAZ,Rainbow (Kim So...",3.0,3.0,2.0,0.0,9.0,6.0,84244.0,76523.0,18.0,10.0,0.0,2.0,1.0,1.0,12.16,LLA/2023 Season/Opening Promotion_Qualifiers_2_3,LLA/2023 Season/Opening Promotion_Qualifiers_2,None,0,LLA
6,LLA/2023 Season/Opening Promotion,The Kings,Globant Emerald,The Kings,Globant Emerald,2022-09-11 23:13:00,3,1,1,27:32,27.533333,"Lucian,Sivir,Yuumi,Braum,Senna","Trundle,Azir,Jarvan IV,Taliyah,Viktor","Renekton,Wukong,Vex,Kalista,Nautilus","Sejuani,Vi,Sylas,Aphelios,Tahm Kench","Destroy (Yoon Jeong-min),MATAZ,Rainbow (Kim So..."